In [1]:
%load_ext autoreload
%autoreload 2

from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,balanced_accuracy_score,f1_score
from answer_judge.vqaEval import VQAEval
import json

/root/miniconda3/envs/VL-Uncertainty/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('/home/hallscope/VL_Uncertainty/exp/log_2025_02_01_18_42_13_gen.json', 'r') as file: 
    data = json.load(file)

In [3]:
df=pd.DataFrame(data)
df = df.drop(columns=['args','begin_time_str','Total samples','end_time_str'])

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
question,Is there a toilet in the image?\nAnswer the qu...,Is there a person in the image?\nAnswer the qu...,Is there a sink in the image?\nAnswer the ques...,Is there an oven in the image?\nAnswer the que...,Is there a bottle in the image?\nAnswer the qu...,Is there a dining table in the image?\nAnswer ...,Is there a person in the image?\nAnswer the qu...,Is there a car in the image?\nAnswer the quest...,Is there a skateboard in the image?\nAnswer th...,Is there a backpack in the image?\nAnswer the ...,...,Is there a chair in the image?\nAnswer the que...,Is there a bottle in the image?\nAnswer the qu...,Is there a cup in the image?\nAnswer the quest...,Is there a bowl in the image?\nAnswer the ques...,Is there a chair in the image?\nAnswer the que...,Is there a couch in the image?\nAnswer the que...,Is there a banana in the image?\nAnswer the qu...,Is there an apple in the image?\nAnswer the qu...,Is there a person in the image?\nAnswer the qu...,Is there a car in the image?\nAnswer the quest...
gt_ans,1,0,1,0,1,0,1,0,1,0,...,1,0,1,0,1,0,1,0,1,0
flag_sample_valid,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
ans,Yes,No,Yes,No,Yes,No,Yes,No,Yes,No,...,Yes,No,Yes,No,Yes,No,Yes,No,Yes,No
perturbed_question_list,[Does the image contain a toilet?\nAnswer the ...,[Is anyone present in the image?\nAnswer the q...,[Does the image contain a sink?\nAnswer the qu...,[Does the image contain an oven?\nAnswer the q...,[Does the image contain a bottle?\nAnswer the ...,[Does the image contain a dining table?\nAnswe...,[Is anyone present in the image?\nAnswer the q...,[Does the image contain a vehicle?\nAnswer the...,[Does the image contain a skateboard?\nAnswer ...,[Does the image contain a backpack?\nAnswer th...,...,[Does the image contain a chair?\nAnswer the q...,[Does the image contain a bottle?\nAnswer the ...,[Does the image contain a cup?\nAnswer the que...,[Does the image contain a bowl?\nAnswer the qu...,[Does the image contain a chair?\nAnswer the q...,[Does the image contain a couch?\nAnswer the q...,[Does the image contain a banana?\nAnswer the ...,[Does the image contain an apple?\nAnswer the ...,[Is anyone present in the image?\nAnswer the q...,[Does the image contain a vehicle?\nAnswer the...


In [ ]:
df.shape

(6, 500)

In [6]:
i=1

In [7]:
df[str(i)]

question                   Is there a person in the image?\nAnswer the qu...
gt_ans                                                                     0
flag_sample_valid                                                       True
ans                                                                       No
perturbed_question_list    [Is anyone present in the image?\nAnswer the q...
ans_sampling_list                                       [No, No, No, No, No]
Name: 1, dtype: object

In [ ]:
df[str(i)]['gt_ans']

0

In [9]:
df[str(i)]['ans']

'No'

In [10]:
df.loc['gt_ans']

0      1
1      0
2      1
3      0
4      1
      ..
495    0
496    1
497    0
498    1
499    0
Name: gt_ans, Length: 500, dtype: object

In [13]:
def get_ans(x):
    return x

gt_label=df.loc['gt_ans'].apply(get_ans).to_list()
gt_label[:5]

[1, 0, 1, 0, 1]

In [14]:
rsps=df.loc['ans'].to_list()
#rsps

In [16]:
is_correct=[]
for i,x in enumerate(rsps):
    if x.lower()=='no' and gt_label[i]==0:
        is_correct.append(1)
    elif x.lower()=='yes' and gt_label[i]==1:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [17]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 500 responses. 434 answer correctly and 66 answer wrongly


In [18]:
%load_ext autoreload
%autoreload 2
from VL_Uncertainty import uncertainty_estimation,hallucination_detection,obtain_lvlm,obtain_llm

class Args: 
    def __init__(self): 
        self.uncertainty_thres=0.8
        self.benchmark = 'open_end'
        self.llm='Qwen2.5-3B'
        self.image_shape=None
        self.sampling_time=5
        
args=Args()

llm=obtain_llm(args)
uncertainty=[]
detection_flag=[]
detect_preds=[]

num_samples=2000
for i in tqdm(range(int(data['Total samples']))):
    idx=str(i)
    data[idx]['flag_ans_correct']=True if is_correct[i]==1 else False
    uncertainty_estimation(args,idx,llm,data)
    uncertainty.append(data[idx]['uncertainty'])
    hallucination_detection(args,idx,data)
    detection_flag.append(data[idx]['flag_detection_correct'])
    detect_preds.append(data[idx]['flag_predict_hallucination'])
    if i >=num_samples:
        break

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


  0%|          | 0/500 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
  0%|          | 1/500 [00:01<11:21,  1.37s/it]Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_i

In [19]:
filename='exp/POPE_Adv_checked.json'
with open(filename, 'w') as file: 
    json.dump(data, file)

In [20]:
import numpy as np

In [21]:
detect_preds=1-np.array(detect_preds)

In [22]:
detect_preds.shape

(500,)

In [23]:
uncertainty=-np.array(uncertainty)

In [24]:
print(f'acc: {accuracy_score(is_correct[:num_samples+1],detect_preds)} f1:{f1_score(is_correct[:num_samples+1],detect_preds)} auroc:{roc_auc_score(is_correct[:num_samples+1],uncertainty)} balanced_accuracy_score:{balanced_accuracy_score(is_correct[:num_samples+1],detect_preds)}')

acc: 0.856 f1:0.9198218262806236 auroc:0.6586894288507192 balanced_accuracy_score:0.5894428152492669
